<h1> In this notebook, we will implement a neural network that regresses volume against segmentation + depth map </h1>

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/root/alok/github/cv_research/alok')

import glob
import json
import os
import numpy as np
from PIL import Image

import keras
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K
from keras.backend import tf as ktf

from matplotlib import pyplot as plt
from matplotlib import cm
# from biomass_utils.points_of_interest import get_point_cloud

%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]="0"
np.random.seed(1)

In [ ]:
# Get data ready

data_path_base = '/root/data/blender_v3'
image_dir = '{}/{}'.format(data_path_base, 'stereo_images')
depth_map_dir = '{}/{}'.format(data_path_base, 'depth_map')
segmentation_dir = '{}/{}'.format(data_path_base, 'mask')
annotation_dir = '{}/{}'.format(data_path_base, 'annotations')

number_key = lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1])
side_key = lambda x: os.path.splitext(os.path.basename(x))[0].split('_')[0]
all_image_paths = sorted(glob.glob('{}/*.png'.format(image_dir)), key=number_key)
image_paths = [p for p in all_image_paths if side_key(p) == 'left'] # we are working with left frame only
all_segmentation_paths = sorted(glob.glob('{}/*.npy'.format(segmentation_dir)), key=number_key)
segmentation_paths = [p for p in all_segmentation_paths if side_key(p) == 'left'] # we are working with left frame only
depth_map_paths = sorted(glob.glob('{}/*.npy'.format(depth_map_dir)), key=number_key)
annotation_paths = sorted(glob.glob('{}/*.json'.format(annotation_dir)), key=number_key) 
complete_data_list = zip(image_paths, segmentation_paths, depth_map_paths, annotation_paths)


TRAINING_SIZE = 500
train_data_list = [v for i, v in enumerate(complete_data_list) if i < TRAINING_SIZE]
test_data_list = [v for i, v in enumerate(complete_data_list) if i > TRAINING_SIZE]


In [ ]:
sm = cm.ScalarMappable(cmap='hot')
sm.set_clim(vmin=0, vmax=20)

def convert_to_gradient_image(image_array, segmentation_mask, depth_map, annotation):    
    projected_depth_map = get_point_cloud(depth_map, annotation['focal_length'], 
                                          annotation['sensor_height'], 
                                          annotation['sensor_width'])[:,:,1]
    cleaned_depth_map = segmentation_mask * projected_depth_map
    rgb_depth_map = sm.to_rgba(cleaned_depth_map, bytes=True)[:,:,:3]
    image_array = image_array
    image_array = np.dstack([image_array, np.ones((image_array.shape[0], image_array.shape[1], 1))*255]).astype('uint8')
    rgb_depth_map = np.dstack([rgb_depth_map, np.ones((rgb_depth_map.shape[0], rgb_depth_map.shape[1], 1))*170]).astype('uint8')
    composite_image = Image.alpha_composite(Image.fromarray(image_array), Image.fromarray(rgb_depth_map))
    composite_image = composite_image * np.dstack([segmentation_mask, segmentation_mask, segmentation_mask, segmentation_mask])
    composite_image = composite_image[:,:,:3]
    return Image.fromarray(composite_image).resize((224, 224))
    
        


In [ ]:
idx = 29
a = convert_to_gradient_image(np.array(Image.open(image_paths[idx]))[:,:,:3],
                                 np.load(segmentation_paths[idx]),
                                 np.load(depth_map_paths[idx]).T, 
                                 json.load(open(annotation_paths[idx], 'rb')))


In [ ]:
def generator(data_list, steps_per_epoch, BATCH_SIZE, INPUT_SHAPE):
    i = 0
    while True:
        x_batch = np.empty((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
        y_batch = np.empty((BATCH_SIZE, 1))
        for (ind, j) in enumerate(range(i*BATCH_SIZE, min((i+1)*BATCH_SIZE, len(data_list)))):
            image_array = np.array(Image.open(data_list[j][0]))[:,:,:3]
            segmentation_mask = np.load(data_list[j][1])
            depth_map = np.load(data_list[j][2]).T
            annotation = json.load(open(data_list[j][3], 'rb'))
            four_channel_input = convert_to_gradient_image(image_array, segmentation_mask, depth_map, annotation)
            x_batch[ind, ...] = four_channel_input
            y_batch[ind] = annotation['volume']
            print(data_list[j])
            
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
# Generate a model with all layers (with top)
vgg16 = VGG16(weights=None, include_top=True)

#Add a layer where input is the output of the  second last layer 
x = Dense(1, name='predictions')(vgg16.layers[-2].output)

#Then create the corresponding model 
model = Model(input=vgg16.input, output=x)
model.summary()

In [ ]:
optimizer = keras.optimizers.adam(lr=0.0001, decay=0.1)
model.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 25
steps_per_epoch = int(len(train_data_list)/BATCH_SIZE)
gen = generator(train_data_list, steps_per_epoch, BATCH_SIZE, (224, 224, 3))

In [ ]:
model.fit_generator(gen, steps_per_epoch=steps_per_epoch, epochs=100, verbose=True)

In [ ]:
eval_gen = generator(test_data_list, np.inf, BATCH_SIZE, (224, 224, 3))
scores = model.evaluate_generator(eval_gen, 50)

In [ ]:
eval_gen = generator(test_data_list, np.inf, BATCH_SIZE, (224, 224, 3))
predictions = model.predict_generator(eval_gen, 120)

In [ ]:
ground_truth_values = np.array([])
for i in range(120*25):
    annotation = json.load(open(test_data_list[i][3], 'rb'))
    ground_truth_values = np.append(ground_truth_values, annotation['volume'])

In [ ]:
(ground_truth_values.mean() - predictions[:,0].mean())/(ground_truth_values.mean())

In [ ]:
ground_truth_values

In [ ]:
predictions[:,0]

In [ ]:
ground_truth_values.mean()

In [ ]:
ground_truth_values

In [ ]:
plt.scatter(ground_truth_values, predictions[:,0])

In [ ]:
np.percentile((abs(ground_truth_values - predictions[:,0])/ground_truth_values), 95)

In [ ]:
json.load(open(annotation_paths[0]))